# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Adtividad Semana 5

### **Vectores Embebidos de OpenAI**

#### **Nombres y matrículas de los integrantes del equipo:**



*   Elemento de lista
*   Carlos Pano Hernandez - A01066264



In [48]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:

import pandas as pd  
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import re
import string

from openai import OpenAI
from dotenv import load_dotenv
import os

from sklearn.model_selection import train_test_split

from collections import Counter

#import spacy
# pip install -U spacy
# python -m spacy download en_core_web_lg
#import en_core_web_lg
#from scipy.spatial import distance


#import fasttext
#import fasttext.util

import openai
import pickle
from tqdm import tqdm
#from tenacity import retry, wait_random_exponential, stop_after_attempt


In [22]:
# Descargamos los paquetes necesarios
nltk.download('punkt')   
nltk.download('stopwords')
negwords = [ 'no', 'nor', 'not', 'ain', 'aren', "aren't", 'don', "don't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

mystopwords = set(stopwords.words('english')) - set(negwords)
print("\nStop words excluding negation words:")
print("-" * 40)
print(sorted(mystopwords))
print("-" * 40)


Stop words excluding negation words:
----------------------------------------
['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'd', 'did', 'do', 'does', 'doing', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'has', 'have', 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', 'it', "it'd", "it'll", "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'more', 'most', 'my', 'myself', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'she', "she'd", "she'll", "she's", 'should', "should've", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', "the

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/charliepano/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/charliepano/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
# Download the English fasttext model
#fasttext.util.download_model('en', if_exists='ignore')  # English
#fasttext.util.download_model('es', if_exists='ignore')  # Español
#cc_en = fasttext.load_model('cc.en.300.bin')


In [23]:
# Incluye las celdas necesarias para tu acceso a la API de OpenAI.

# Load environment variables from the specific .env file -> Amigos, aqui cada quien configuere su propia API key en el .env
# El .gitignore ya excluye el mismo.

load_dotenv('.env')

api_key = os.getenv('OPENAI_API_KEY')
if api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable is not set")

client = OpenAI(api_key=api_key)

# Prueba de la API de OpenAI
response = client.responses.create(
    model="gpt-4o-mini",
    input="Hola, como estas? (Responde en 1 palabra y en Aleman)"
)

print(response.output_text)


Gut.


# **Pregunta - 1:**



Descarga los 3 archivos de Canvas y genera un nuevo DataFrame de Pandas con ellos.

**Llama simplemente "df" a dicho DataFrame.**

Los archivos los encuentras en Canvas: amazon5.txt, imdb5.txt, yelp5.txt.



In [24]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

# Reding the three text files
amazon_df = pd.read_csv('Data/amazon5.txt', sep='\t', header=None, names=['text', 'label'])
imdb_df = pd.read_csv('Data/imdb5.txt', sep='\t', header=None, names=['text', 'label']) 
yelp_df = pd.read_csv('Data/yelp5.txt', sep='\t', header=None, names=['text', 'label'])

# Add source column to identify which dataset each review came from
amazon_df['source'] = 'amazon'
imdb_df['source'] = 'imdb'
yelp_df['source'] = 'yelp'

# Concatenate
df = pd.concat([amazon_df, imdb_df, yelp_df], ignore_index=True)

# *********** Aquí termina la sección de agregar código *************


In [25]:
# Verifiquemos la información del DataFrame:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    3000 non-null   object 
 1   label   2000 non-null   float64
 2   source  3000 non-null   object 
dtypes: float64(1), object(2)
memory usage: 70.4+ KB


In [26]:
# Y veamos sus primeros registros:

print(df.head())

X = df.text   
Y = df.label    

# Verificamos que las dimensiones sean las esperadas (3,000 reviews)
assert X.shape == (3000,)          
assert Y.shape == (3000,)

                                                text  label  source
0  So there is no way for me to plug it in here i...    0.0  amazon
1                        Good case, Excellent value.    1.0  amazon
2                             Great for the jawbone.    1.0  amazon
3  Tied to charger for conversations lasting more...    0.0  amazon
4                                  The mic is great.    1.0  amazon


# **Pregunta - 2:**

Realiza el proceso de limpieza. Aplica el preprocesamiento que consideres adecuado.











In [27]:
# Text preprocessing functions

def clean_tok(doc):
    # Basic cleaning and tokenization
    doc = doc.lower()
    
    # Remove non-letters
    doc = re.sub(r'[^a-z\s]', ' ', doc)
    
    # Split into words
    tokens = doc.split()
    
    # Filter stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Remove single chars
    tokens = [token for token in tokens if len(token) > 1]

    return tokens


def clean_doc(doc):
    # Token normalization
    ps = PorterStemmer()
    wnl = WordNetLemmatizer()
    
    tokens = []
    
    for token in doc:
        # Clean repeating chars
        token = re.sub(r'(.)\1{2,}', r'\1\1', token)
        token = ps.stem(token)
        token = wnl.lemmatize(token)
        tokens.append(token)
        
    return tokens

# Clean and tokenize
Xclean = [clean_tok(x) for x in X]   
print("Tokens after cleaning:")
print(*Xclean[0:5], sep='\n')

# Normalize tokens
Xclean = [clean_doc(x) for x in Xclean]  
print("\nTokens after normalization:")
print(*Xclean[0:5], sep='\n')

Tokens after cleaning:
['way', 'plug', 'us', 'unless', 'go', 'converter']
['good', 'case', 'excellent', 'value']
['great', 'jawbone']
['tied', 'charger', 'conversations', 'lasting', 'minutes', 'major', 'problems']
['mic', 'great']

Tokens after normalization:
['way', 'plug', 'u', 'unless', 'go', 'convert']
['good', 'case', 'excel', 'valu']
['great', 'jawbon']
['tie', 'charger', 'convers', 'last', 'minut', 'major', 'problem']
['mic', 'great']


In [28]:
# Despleguemos los primeros comentarios después de tu proceso de limpieza:

for x in Xclean[0:5]:
  print(x)


['way', 'plug', 'u', 'unless', 'go', 'convert']
['good', 'case', 'excel', 'valu']
['great', 'jawbon']
['tie', 'charger', 'convers', 'last', 'minut', 'major', 'problem']
['mic', 'great']


# **Pregunta - 3:**



Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba, respectivamente.

In [29]:

# ************* Inicia la sección de agregar código:*****************************

x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(Xclean, Y, train_size=.70, shuffle=True, random_state=1) 
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=.50, shuffle=True, random_state=17)


# *********** Termina la sección de agregar código *************


# verificemos las dimensiones obtenidas:
print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


# **Pregunta - 4:**



Construye tu vocabulario a continuación


In [30]:
# a.	Usa el conjunto de entrenamiento para generar tu vocabulario
#     con un tamaño que consideres adecuado:


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


midiccionario = Counter()    

for k in range(len(x_train)):
  midiccionario.update(x_train[k])


# *********** Aquí termina la sección de agregar código *************

In [31]:
# b.	Indica el tamaño del vocabulario generado.

# ******* Inicia la sección de agregar código: ***********


print('Longitud del diccionario:', len(midiccionario))  
print('\n(word,frequency):') 
print(midiccionario.most_common(10)) 


# *********** Aquí termina la sección de agregar código *************

Longitud del diccionario: 3145

(word,frequency):
[('good', 164), ('movi', 140), ('great', 139), ('phone', 134), ('film', 130), ('work', 113), ('time', 102), ('like', 101), ('one', 100), ('place', 87)]


c.	¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el vocabulario?


### ++++++++ Inicia la sección de agregar texto: +++++++++++

Existen varias razones, entre las principales se encuentran:

1. **Evitar data leakage:** Si usáramos otros conjuntos para nuestro diccionario, estaríamos filtrando información que debería ser completamente desconocida durante el entrenamiento.
2. **Simular escenarios reales:** En un caso real, no tendremos acceso a datos futuros.
3. **Detectar overfitting:** Si el vocabulario se construyera con todos los datos, sería más difícil detectar si el modelo está memorizando en lugar de aprendiendo patrones generalizables.

### ++++++++ Termina la sección de agregar texto: +++++++++++


In [32]:
# d.	Con el vocabulario generado, filtra los conjuntos de entrenamiento,
#     validación y prueba para que todos los comentarios usen solamente las
#     palabras de este vocabulario.

#     Llamar train_x, val_x y test_x a estos tres conjuntos.


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

# Crear conjuntos filtrados usando solo palabras del vocabulario
train_x = []
val_x = []
test_x = []

# Filtrar conjunto de entrenamiento
for comment in x_train:
    filtered_comment = [word for word in comment if word in midiccionario]
    train_x.append(filtered_comment)

# Filtrar conjunto de validación  
for comment in x_val:
    filtered_comment = [word for word in comment if word in midiccionario]
    val_x.append(filtered_comment)

# Filtrar conjunto de prueba
for comment in x_test:
    filtered_comment = [word for word in comment if word in midiccionario]
    test_x.append(filtered_comment)

# *********** Aquí termina la sección de agregar código *************


In [33]:
# Vemos el resultado de los primeros comentarios del conjunto de entrenamiento:

for ss in train_x[0:5]:
  print(ss)

['co', 'star', 'fare', 'much', 'better', 'peopl', 'like', 'morgan', 'freeman', 'jonah', 'hill', 'ed', 'helm', 'wast']
['tonight', 'elk', 'filet', 'special', 'suck']
['paid', 'bill', 'tip', 'felt', 'server', 'terribl', 'job']
['call', 'steakhous', 'properli', 'cook', 'steak', 'understand']
['howev', 'keypad', 'tinni', 'sometim', 'reach', 'wrong', 'button']


# **Pregunta - 5:**


#### **Incluye aquí un resumen de las características y diferencias que tiene al menos los tres modelos de OpenAI indicados: "text-embedding-3-small", "text-embedding-3-large" y "text-embedding-ada-002".**

### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++


# **Pregunta - 6:**


#### **Diccionario clave-valor de palabras del diccionario y vectores embebidos.**

In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

openai.api_key = os.getenv("OPENAI_API_KEY")

# Seleccionamos el modelo de embeddings (usando el más económico de los mencionados)
EMBEDDING_MODEL = "text-embedding-3-small"
EMBEDDING_ENCODING = "cl100k_base"  # Este es el encoding recomendado para el modelo
EMBEDDING_DIMENSIONS = 1536  # Dimensiones del modelo small

# Función con reintentos para manejar errores de API
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(3))
def get_embedding(text):
    response = openai.embeddings.create(
        model=EMBEDDING_MODEL,
        input=text,
        encoding_format="float"
    )
    return response.data[0].embedding

# Verificamos si el archivo ya existe
if not os.path.exists('vocabulary_embeddings.pkl'):
    embeddings_dict = {}
    total_tokens = 0

    # Generamos embeddings para cada palabra en el vocabulario
    print("Generando embeddings para el vocabulario...")
    for word in tqdm(midiccionario):
        embedding = get_embedding(word)
        embeddings_dict[word] = embedding
        # Aproximadamente 1 token por palabra para palabras simples
        total_tokens += 1

    print(f"\nTotal de tokens utilizados: {total_tokens}")
    print(f"Dimensiones del vector de embeddings: {EMBEDDING_DIMENSIONS}")
    print(f"Tamaño del vocabulario procesado: {len(embeddings_dict)}")

    # Guardamos el diccionario de embeddings
    with open('vocabulary_embeddings.pkl', 'wb') as f:
        pickle.dump(embeddings_dict, f)

    print("\nDiccionario de embeddings guardado en 'vocabulary_embeddings.pkl'")

    # Mostramos un ejemplo del diccionario
    sample_word = list(embeddings_dict.keys())[0]
    print(f"\nEjemplo de embedding para la palabra '{sample_word}':")
    print(f"Dimensiones del vector: {len(embeddings_dict[sample_word])}")
    print(f"Primeros 5 valores del vector: {embeddings_dict[sample_word][:5]}")
else:
    # Cargamos el diccionario de embeddings existente
    with open('vocabulary_embeddings.pkl', 'rb') as f:
        embeddings_dict = pickle.load(f)
        
    print("El archivo vocabulary_embeddings.pkl ya existe. Omitiendo la generación de embeddings.")
    
    # Mostramos un ejemplo del diccionario
    sample_word = list(embeddings_dict.keys())[0]
    print(f"\nEjemplo de embedding para la palabra '{sample_word}':")
    print(f"Dimensiones del vector: {len(embeddings_dict[sample_word])}")
    print(f"Primeros 5 valores del vector: {embeddings_dict[sample_word][:5]}")


# *********** Aquí termina la sección de agregar código *************

El archivo vocabulary_embeddings.pkl ya existe. Omitiendo la generación de embeddings.

Ejemplo de embedding para la palabra 'co':
Dimensiones del vector: 1536
Primeros 5 valores del vector: [0.04341873, 0.0015039851, 0.0046208845, 0.042929508, 0.029063033]


# **Pregunta - 7:**



Generamos los vectores embebidos a partir de los conjuntos de entrenamiento, validación y prueba.

Los llamaremos trainEmb, valEmb y testEmb, respectivamente.

In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

In [ ]:
# Veamos las dimensiones de cada conjunto embebido:

print("Train-Emb:", trainEmb.shape)
print("Val-Emb:", valEmb.shape)
print("Test-Emb:", testEmb.shape)

# **Pregunta - 8:**



Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños.

Compara los resultados con los de la semana anterior.

In [ ]:
# REGRESIÓN LOGÍSTICA:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************


In [ ]:
# BOSQUE ALEATORIO (Random Forest):

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

# **Pregunta - 9:**



Reporte del mejor modelo con el conjunto de Prueba (Test).


In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

# **Pregunta - 10:**

In [ ]:
# Incluye todas las líneas de código y celdas que consideres adecuadas para este ejercicio.


None




# **Pregunta - 11:**



Incluye tus comentarios finales de la actividad.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++

# **Fin de la Actividad de Vectores Embebidos - OpenAI**